In [ ]:
# default_exp tspecscores


# tspecscores

> Calculate different tissue specificity metrics

In [ ]:
# hide
from nbdev.showdoc import *


In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# export
import pandas as pd
import numpy as np


In [ ]:
# export
def log_it(df: pd.DataFrame) -> pd.DataFrame:
    return np.log(1 + df)


In [ ]:
# export
def tsi(df: pd.DataFrame) -> pd.DataFrame:
    foo = np.sum(df.values, axis=1)[:, None]
    return pd.DataFrame(
        np.divide(df.values, foo, where=foo != 0), index=df.index, columns=df.columns
    )


In [ ]:
# export
def spm(df: pd.DataFrame) -> pd.DataFrame:
    foo = np.linalg.norm(df.values, axis=1)[:, None] * df.values
    return pd.DataFrame(
        np.divide(df.values ** 2, foo, where=foo != 0),
        index=df.index,
        columns=df.columns,
    )


In [ ]:
# export
def zscore(df: pd.DataFrame, transform=True) -> pd.DataFrame:
    std = np.std(df.values, axis=1, ddof=1)[:, None]
    zs = np.divide(df.values - df.values.mean(axis=1)[:, None], std, where=std != 0)
    zs[np.where(std == 0)[0], :] = 0
    if transform:
        max_zs = (df.values.shape[1] - 1) / np.sqrt(df.values.shape[1])
        zs = np.divide((zs + max_zs), (2 * max_zs), where=zs != 0)
    return pd.DataFrame(zs, index=df.index, columns=df.columns)



In [ ]:
# export
def tau(df):
    foo = np.max(df.values, axis=1)[:, None]
    vector_r = np.divide(df.values, foo, where=foo != 0)
    tau_index = np.sum(1 - vector_r, axis=1) / (df.values.shape[1] - 1)
    return pd.Series(tau_index, index=df.index)


In [ ]:
# export
def calc_ts(
    df: pd.DataFrame, method: str = "spm", log: bool = True, **kwargs
) -> pd.DataFrame:
    ts_func = {"tsi": tsi, "spm": spm, "zscore": zscore, "tau": tau}
    ts_df = df.copy()
    if log:
        ts_df = log_it(ts_df)

    return ts_func[method](ts_df, **kwargs)
